In [0]:
!pip install xlrd
!pip install openpyxl
!pip install pydotplus
!pip install imbalanced-learn

Requirement already satisfied: xlrd in /databricks/python3/lib/python3.8/site-packages (1.2.0)
 WARNING: You are using pip version 21.0.1; however, version 22.0.4 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command. 
Requirement already satisfied: openpyxl in /databricks/python3/lib/python3.8/site-packages (3.0.9)
Requirement already satisfied: et-xmlfile in /databricks/python3/lib/python3.8/site-packages (from openpyxl) (1.1.0)
 WARNING: You are using pip version 21.0.1; however, version 22.0.4 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command. 
Requirement already satisfied: pydotplus in /databricks/python3/lib/python3.8/site-packages (2.0.2)
Requirement already satisfied: pyparsing>=2.0.1 in /databricks/python3/lib/python3.8/site-packages (from pydotplus) (2.4.7)
 WARNING: You are using pip version 21.0.1; however, version 22.0.4 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command. 
Requirement already satisfied: imbalanced-learn in /databricks/python3/lib/python3.8/site-packages (0.9.0)
Requirement already satisfied: threadpoolctl>=2.0.0 in /databricks/python3/lib/python3.8/site-packages (from imbalanced-learn) (2.1.0)
Requirement already satisfied: numpy>=1.14.6 in /databricks/python3/lib/python3.8/site-packages (from imbalanced-learn) (1.20.1)
Requirement already satisfied: joblib>=0.11 in /databricks/python3/lib/python3.8/site-packages (from imbalanced-learn) (1.0.1)
Requirement already satisfied: scikit-learn>=1.0.1 in /databricks/python3/lib/python3.8/site-packages (from imbalanced-learn) (1.0.2)
Requirement already satisfied: scipy>=1.1.0 in /databricks/python3/lib/python3.8/site-packages (from imbalanced-learn) (1.6.2)
 WARNING: You are using pip version 21.0.1; however, version 22.0.4 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.

In [0]:
import pandas as pd
import numpy as np
from scipy.stats import norm

import os
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns


import pickle
from datetime import datetime as dt
from datetime import timedelta
from pandas.tseries.offsets import BDay
import math
import calendar
import datetime
from datetime import date, timedelta

from sklearn import tree
import pydotplus
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier,RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.image as pltimg
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.combine import SMOTEENN

from imblearn.under_sampling import EditedNearestNeighbours
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
import pickle
import os

%matplotlib inline

In [0]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [0]:
model_run_date='2022-03-25'

h_inv_data='Historical_inventory_rawData_169_03_01'
#reading historical inventory data
h_inv = spark.table(h_inv_data)
h_inv=h_inv.toPandas()


Mb51_data='consumptionmb51_roh_zpck_28_12'
# MB51 import data
Mb51 = spark.table(Mb51_data)
Mb51 = Mb51.toPandas()

ves_data='model_data_vf_19_01'
#ves data import
ves = spark.table(ves_data)
ves = ves.toPandas()
ves = ves[ves['PO_Create_Date']!='1888-01-01']
ves = ves[ves['Delivered_Quantity']!=0]

ves_w_po='enter file path to VES_data_final_06_12.csv'
#reading ves old data to get the planned lead time
ves_old=pd.read_csv(ves_w_po)

In [0]:
h_inv_consumption_table="h_inv_ves_consump_02_feb"

**To identify the best ML based order placement model and use the same for forecasting order placement over the next 6 months.
**Inorder to create a model to predict the reorder points we need the following:
  1. Inventory in hand (Historical Inventory data)
  2. MB51 consumption data
  3. VES data
  4. VES data with planned lead time

In [0]:
#getting required fields from inventory
h_inv_f=h_inv[['SnapshotDate','MaterialID','LocationID','BatchID','InventoryStockUnRestricted','InventoryUnitOfMeasure']]
h_inv_f['SnapshotDate']=pd.to_datetime(h_inv['SnapshotDate'],format="%Y-%m-%d")

# Creating Week attribute 
h_inv_f['Week']=h_inv_f['SnapshotDate'].dt.week
h_inv_f['SnapshotDate']=pd.to_datetime(h_inv['SnapshotDate'],format="%Y-%m-%d").dt.date

In [0]:
#To check if material has more than one UoM
dat_check=h_inv_f[['MaterialID','InventoryUnitOfMeasure']]
dat_check.drop_duplicates(inplace=True)

dat_check1=pd.pivot_table(dat_check,values='InventoryUnitOfMeasure',index=['MaterialID'],aggfunc='count')
print(len(dat_check1['InventoryUnitOfMeasure'].unique()))
dat_check1['InventoryUnitOfMeasure'].value_counts()

1
Out[57]: 1 171
Name: InventoryUnitOfMeasure, dtype: int64

In [0]:
#identifying total inventory stock unrestricted across batches for a given plant,material and day
h_inv_f['InventoryStockUnRestricted_daily']=h_inv_f.groupby(by=['LocationID','MaterialID','SnapshotDate'])['InventoryStockUnRestricted'].transform(np.sum)

#dropping batchID to get the plant, material and day wise inventory stock unrestricted
h_inv_f.drop(columns=['BatchID','InventoryStockUnRestricted'],axis=1,inplace=True)
h_inv_f.drop_duplicates(inplace=True)

#sorting value with respect location ,material, snapshotdate and batch
h_inv_fi=h_inv_f.sort_values(by=['LocationID','MaterialID','SnapshotDate']).reset_index()
h_inv_fi.drop(columns='index',axis=1,inplace=True)
h_inv_fi['SnapshotDate']=pd.to_datetime(h_inv_fi['SnapshotDate'],format="%Y-%m-%d")
print(h_inv_fi.shape)
h_inv_fi.head()

(303359, 6)
Out[58]:

,SnapshotDate,MaterialID,LocationID,InventoryUnitOfMeasure,Week,InventoryStockUnRestricted_daily
0,2019-11-30,22000213,5004,LB,48,2325.33
1,2019-12-01,22000213,5004,LB,48,2325.33
2,2019-12-02,22000213,5004,LB,49,2325.33
3,2019-12-03,22000213,5004,LB,49,2043.23
4,2019-12-04,22000213,5004,LB,49,2043.23


In [0]:
#Code to fill MB51 missing dates
Mb51['Posting_Date'] = pd.to_datetime(Mb51['Posting_Date'])
Mb51matls = Mb51[['Plant', 'Material']].drop_duplicates()

#Populating missing dates
Mb51alldates = pd.DataFrame()
for plant, matl in zip(Mb51matls['Plant'], Mb51matls['Material']):
  Start = min(Mb51[(Mb51['Plant'] == plant) & (Mb51['Material'] == matl)]['Posting_Date'])
  End = max(Mb51[(Mb51['Plant'] == plant) & (Mb51['Material'] == matl)]['Posting_Date'])

  dRan1 = pd.date_range(start =Start, end =End, freq ='D')
  dRan1 = pd.DataFrame({'Plant':plant, 'Material': matl, 'Date':dRan1})
  
  Mb51alldates = Mb51alldates.append(dRan1, ignore_index= True)
  
#merging all dates with MB51
Mb51matls = pd.merge(Mb51alldates, Mb51, left_on = ['Plant', 'Material', 'Date'], right_on = ['Plant', 'Material', 'Posting_Date'], how = 'left')
Mb51matls.drop(['Posting_Date'], inplace = True, axis = 1)

#replacing null values with 0
Mb51matls['Qty_in_BUoM'] = Mb51matls['Qty_in_BUoM'].fillna(0)

#creating week attribute
Mb51matls['week'] = Mb51matls['Date'].dt.week

In [0]:
#Creating inventory consumption data table from MB51  
inv_con1 = pd.pivot_table(Mb51matls,values='Qty_in_BUoM',index=['Plant','Material','week'],aggfunc=['mean', 'std']).reset_index()
inv_con1.columns =[s1 + str(s2) for (s1,s2) in inv_con1.columns.tolist()]
inv_con = inv_con1.rename(columns = {'Plant':'LocationID', 'Material':'MaterialID', 'week': 'Week', 'meanQty_in_BUoM':'daily_mean_consumption', 'stdQty_in_BUoM': 'std_dev_consumption'})

In [0]:
# Getting the Planned lead time
pldt=ves_old[['Plant','Material No.','Sum of Planned Days']].copy()
print('number of planned lead time records less than one',pldt[pldt['Sum of Planned Days']<1].shape[0])

#identifying average planned lead time for a given plant and material
pldt_pivot=pd.pivot_table(pldt,values='Sum of Planned Days',index=['Plant','Material No.'],aggfunc='mean').reset_index()
pldt_pivot['Sum of Planned Days']=round(pldt_pivot['Sum of Planned Days'],0)
print('number of unique materials',len(pldt_pivot['Material No.'].unique()))

#renaming the columns
pldt_pivot.rename(columns={'Plant':'LocationID','Material No.':'MaterialID','Sum of Planned Days':'Planned_avg_lead_time'},inplace=True)
pldt_pivot.head(3)

number of planned lead time records less than one 2400
number of unique materials 2494
Out[61]:

,LocationID,MaterialID,Planned_avg_lead_time
0,5004,22000026,90.0
1,5004,22000120,28.0
2,5004,22000206,28.0


**To create model input data below steps are to followed:
  1. Merge inventory in hand data with VES to find the PO create date
  2. Get planned date for the plant,material combination
  3. Generate Expected consumption for planned leadtime of +2,+4 and +6 weeks.
  4. Finding Ratio of expected consumption in leadtime +2,+4 and +6 weeks to average daily consumption and inventory in hand to average daily consumption

In [0]:
# getting PO create dates for all plant material combination from VES
ves_data=ves[['Plant_ID','Material_No.','PO_Create_Date']].copy()
ves_data.drop_duplicates(inplace=True)
ves_data.head()

Out[62]:

,Plant_ID,Material_No.,PO_Create_Date
0,5259,22019000,2021-04-05
1,5259,22019000,2021-03-19
2,5259,22019000,2021-03-26
4,5259,22019000,2021-03-11
8,5259,22019000,2021-06-14


In [0]:
#1. merging to iventory in hand with ves to find the PO create dates.
h_inv_ves=pd.merge(h_inv_fi,ves_data,left_on=['LocationID','MaterialID','SnapshotDate'],right_on=['Plant_ID','Material_No.','PO_Create_Date'],how='left')
h_inv_ves.drop(columns=['Material_No.','PO_Create_Date'],inplace=True)
print(h_inv_ves.shape)

#2. merging inventory at hand with planned lead time
h_inv_ves=pd.merge(h_inv_ves,pldt_pivot,on=['LocationID','MaterialID'],how='inner')
print(h_inv_ves.shape)

# getting planned_gr date with extended_gr date of 2,4 and 6 weeks
h_inv_ves['Planned_GR_date'] = pd.to_datetime(h_inv_ves['SnapshotDate']).dt.date + pd.to_timedelta(h_inv_ves['Planned_avg_lead_time'], unit='D')
h_inv_ves['Planned_GR_date']=pd.to_datetime(h_inv_ves['Planned_GR_date']).dt.date
h_inv_ves['Planned_GR_date']=pd.to_datetime(h_inv_ves['Planned_GR_date'])
h_inv_ves['Extended_GR_date_2']=pd.to_datetime(h_inv_ves['Planned_GR_date'])+timedelta(14)
h_inv_ves['Extended_GR_date_2']=pd.to_datetime(h_inv_ves['Extended_GR_date_2']).dt.date
h_inv_ves['Extended_GR_date_2']=pd.to_datetime(h_inv_ves['Extended_GR_date_2'])
h_inv_ves['Extended_GR_date_4']=pd.to_datetime(h_inv_ves['Planned_GR_date'])+timedelta(28)
h_inv_ves['Extended_GR_date_4']=pd.to_datetime(h_inv_ves['Extended_GR_date_4']).dt.date
h_inv_ves['Extended_GR_date_4']=pd.to_datetime(h_inv_ves['Extended_GR_date_4'])
h_inv_ves['Extended_GR_date_6']=pd.to_datetime(h_inv_ves['Planned_GR_date'])+timedelta(42)
h_inv_ves['Extended_GR_date_6']=pd.to_datetime(h_inv_ves['Extended_GR_date_6']).dt.date
h_inv_ves['Extended_GR_date_6']=pd.to_datetime(h_inv_ves['Extended_GR_date_6'])
h_inv_ves.head()

In [0]:
# getting average consumption for a plant, material combination
avg_consump_w_0=pd.pivot_table(Mb51matls,values='Qty_in_BUoM',index=['Plant','Material'],aggfunc='mean').reset_index()

# to filter those plant, material combination which are available in both consumption and inventory in hand
h_inv_ves_consump=pd.merge(h_inv_ves,avg_consump_w_0[['Plant','Material']],left_on=['LocationID','MaterialID'],right_on=['Plant','Material'],how='inner')
print(h_inv_ves_consump.shape)

#3. Generate Expected consumption for planned leadtime of +2,+4 and +6 weeks.
counters=[]
order_placed=[]
expected_consump_2=[]
expected_consump_4=[]
expected_consump_6=[]
inv_ratio_w_0=[]
expected_consump_ratio_2_w_0=[]
expected_consump_ratio_4_w_0=[]
expected_consump_ratio_6_w_0=[]
counts=0
camp=0
for plt_id,mat_id,snap_date, avgpldt, pl_gr,ex_gr_2,ex_gr_4,ex_gr_6,inventory in zip(h_inv_ves_consump['LocationID'],h_inv_ves_consump['MaterialID'],h_inv_ves_consump['SnapshotDate'],h_inv_ves_consump['Plant_ID'],h_inv_ves_consump['Planned_GR_date'],h_inv_ves_consump['Extended_GR_date_2'],h_inv_ves_consump['Extended_GR_date_4'],h_inv_ves_consump['Extended_GR_date_6'],h_inv_ves_consump['InventoryStockUnRestricted_daily']):
  
  avg_con_w_0=abs(avg_consump_w_0[(avg_consump_w_0['Plant']==plt_id) & (avg_consump_w_0['Material']==mat_id)]['Qty_in_BUoM'].values[0])
  
  #creating a dataframe with all dates starting from snapshot date to extended_gr_date_6 and populating it with daily mean consumption
  inv_con1=inv_con[(inv_con['LocationID']==plt_id) & (inv_con['MaterialID']==mat_id)]
  alldates=pd.date_range(start=snap_date, end=ex_gr_6)
  tempdf1=pd.DataFrame(alldates,columns=['dates'])
  tempdf1['LocationID']=plt_id
  tempdf1['MaterialID']=mat_id
  tempdf1['Week']=tempdf1['dates'].dt.week
  inv_consumption=pd.merge(tempdf1,inv_con1,on=['LocationID','MaterialID','Week'],how='left') 
  
  if math.isnan(avgpldt) == True:
    counts=counts+1
    counters.append(counts)   
    
    consump_2=inv_consumption[(inv_consumption['LocationID']==plt_id) & (inv_consumption['MaterialID']==mat_id) & (inv_consumption['dates']>=pl_gr) & (inv_consumption['dates']<ex_gr_2)]['daily_mean_consumption'].sum()
    consump_4=inv_consumption[(inv_consumption['LocationID']==plt_id) & (inv_consumption['MaterialID']==mat_id) & (inv_consumption['dates']>=pl_gr) & (inv_consumption['dates']<ex_gr_4)]['daily_mean_consumption'].sum()
    consump_6=inv_consumption[(inv_consumption['LocationID']==plt_id) & (inv_consumption['MaterialID']==mat_id) & (inv_consumption['dates']>=pl_gr) & (inv_consumption['dates']<ex_gr_6)]['daily_mean_consumption'].sum()
    expected_consump_2.append(abs(consump_2))
    expected_consump_4.append(abs(consump_4))
    expected_consump_6.append(abs(consump_6))
    order_placed.append(0)
    expected_consump_ratio_2_w_0.append(abs(consump_2/avg_con_w_0))
    expected_consump_ratio_4_w_0.append(abs(consump_4/avg_con_w_0))
    expected_consump_ratio_6_w_0.append(abs(consump_6/avg_con_w_0))
    
    inv_ratio_w_0.append(abs(inventory/avg_con_w_0))
  else:
    counts=counts+1
    counters.append(counts)
    order_placed.append(1)
    
    consump_2=inv_consumption[(inv_consumption['LocationID']==plt_id) & (inv_consumption['MaterialID']==mat_id) & (inv_consumption['dates']>=pl_gr) & (inv_consumption['dates']<ex_gr_2)]['daily_mean_consumption'].sum()
    consump_4=inv_consumption[(inv_consumption['LocationID']==plt_id) & (inv_consumption['MaterialID']==mat_id) & (inv_consumption['dates']>=pl_gr) & (inv_consumption['dates']<ex_gr_4)]['daily_mean_consumption'].sum()
    consump_6=inv_consumption[(inv_consumption['LocationID']==plt_id) & (inv_consumption['MaterialID']==mat_id) & (inv_consumption['dates']>=pl_gr) & (inv_consumption['dates']<ex_gr_6)]['daily_mean_consumption'].sum()
    
    expected_consump_2.append(abs(consump_2))
    expected_consump_4.append(abs(consump_4))
    expected_consump_6.append(abs(consump_6))
    expected_consump_ratio_2_w_0.append(abs(consump_2/avg_con_w_0))
    expected_consump_ratio_4_w_0.append(abs(consump_4/avg_con_w_0))
    expected_consump_ratio_6_w_0.append(abs(consump_6/avg_con_w_0))
    
    inv_ratio_w_0.append(abs(inventory/avg_con_w_0))
    counts=0
  print(camp)
  camp+=1
    
h_inv_ves_consump['counter']=counters
h_inv_ves_consump['Expected_consumption_btw_gr_dates_2']=expected_consump_2
h_inv_ves_consump['Expected_consumption_btw_gr_dates_4']=expected_consump_4
h_inv_ves_consump['Expected_consumption_btw_gr_dates_6']=expected_consump_6
h_inv_ves_consump['inventory_ratio_with_0']=inv_ratio_w_0

# 4. Finding Ratio of expected consumption in leadtime +2,+4 and +6 weeks to average daily consumption and inventory in hand to average daily consumption
h_inv_ves_consump['Expected_consumption_ratio_2_with_0']=expected_consump_ratio_2_w_0
h_inv_ves_consump['Expected_consumption_ratio_4_with_0']=expected_consump_ratio_4_w_0
h_inv_ves_consump['Expected_consumption_ratio_6_with_0']=expected_consump_ratio_6_w_0
h_inv_ves_consump['order_placed']=order_placed

h_inv_ves_consump.drop(columns=['Plant_ID'],inplace=True)
print(h_inv_ves_consump.shape)
h_inv_ves_consump.head()

In [0]:
#saving the model results as a tabel
col_names = ['_'.join(col.split(' ')) for col in h_inv_ves_consump.columns.tolist()]
h_inv_ves_consump.columns = col_names
h_inv_ves_consump_v1 = spark.createDataFrame(h_inv_ves_consump)
h_inv_ves_consump_v1.write.saveAsTable(h_inv_consumption_table,mode = 'overwrite')

In [0]:
h_inv_ves_consump = spark.table(h_inv_consumption_table)
h_inv_ves_consump=h_inv_ves_consump.toPandas()

In [0]:
h_inv_ves_consump.head(1)

Out[64]:

,SnapshotDate,MaterialID,LocationID,InventoryUnitOfMeasure,Week,InventoryStockUnRestricted_daily,Planned_avg_lead_time,Planned_GR_date,Extended_GR_date_2,Extended_GR_date_4,Extended_GR_date_6,Plant,Material,counter,Expected_consumption_btw_gr_dates_2,Expected_consumption_btw_gr_dates_4,Expected_consumption_btw_gr_dates_6,inventory_ratio_with_0,Expected_consumption_ratio_2_with_0,Expected_consumption_ratio_4_with_0,Expected_consumption_ratio_6_with_0,order_placed
0,2021-05-18,43646226,5259,LB,20,840.0,35.0,2021-06-22,2021-07-06,2021-07-20,2021-08-03,5259,43646226,557,8328.5,30046.642857,39643.642857,1.755475,17.405329,62.793026,82.849332,0


In [0]:
# splitting model data into 2 sets before using it in model.
# a. getting plant material combination which has greater than 5% (35 orders) of order that has been placed.
ves_consump=h_inv_ves_consump[(h_inv_ves_consump['order_placed']==1)][['LocationID','MaterialID','order_placed']].copy()
ves_consump['order_counts']=ves_consump.groupby(by=['LocationID','MaterialID'])['order_placed'].transform(np.sum)
ves_consump_v1=ves_consump[ves_consump['order_counts']>=35]
ves_consump_v1.drop(columns=['order_placed','order_counts'],inplace=True)
ves_consump_v1.drop_duplicates(inplace=True)
h_inv_ves_consump_fin=pd.merge(h_inv_ves_consump,ves_consump_v1,on=['LocationID','MaterialID'],how='inner')
print(h_inv_ves_consump_fin.shape)

#getting plant material combination for the first set
Rop01=h_inv_ves_consump_fin[['LocationID','MaterialID']].copy()
Rop01.drop_duplicates(inplace=True)
print(Rop01.shape)

#first data set
h_inv_ves_consump_fin_v2=h_inv_ves_consump_fin.drop(columns=['Plant','Material'])

print(h_inv_ves_consump_fin_v2.shape)
h_inv_ves_consump_fin_v2.head()

(140544, 22)
(207, 2)
(140544, 20)
Out[65]:

,SnapshotDate,MaterialID,LocationID,InventoryUnitOfMeasure,Week,InventoryStockUnRestricted_daily,Planned_avg_lead_time,Planned_GR_date,Extended_GR_date_2,Extended_GR_date_4,Extended_GR_date_6,counter,Expected_consumption_btw_gr_dates_2,Expected_consumption_btw_gr_dates_4,Expected_consumption_btw_gr_dates_6,inventory_ratio_with_0,Expected_consumption_ratio_2_with_0,Expected_consumption_ratio_4_with_0,Expected_consumption_ratio_6_with_0,order_placed
0,2019-11-30,43647982,5259,LB,48,45446.1,28.0,2019-12-28,2020-01-11,2020-01-25,2020-02-08,729,91200.714286,202102.304286,302728.112857,7.586372,15.224244,33.737179,50.534765,0
1,2019-12-01,43647982,5259,LB,48,45446.1,28.0,2019-12-29,2020-01-12,2020-01-26,2020-02-09,730,96050.082143,205157.622143,306069.321429,7.586372,16.033755,34.247206,51.092517,0
2,2019-12-02,43647982,5259,LB,49,42600.9,28.0,2019-12-30,2020-01-13,2020-01-27,2020-02-10,731,100899.450000,208212.940000,309410.530000,7.111419,16.843265,34.757234,51.650269,0
3,2019-12-03,43647982,5259,LB,49,35349.5,28.0,2019-12-31,2020-01-14,2020-01-28,2020-02-11,732,103609.791429,209763.690714,311628.845000,5.900934,17.295706,35.016103,52.020575,0
4,2019-12-04,43647982,5259,LB,49,68721.2,28.0,2020-01-01,2020-01-15,2020-01-29,2020-02-12,733,106320.132857,211314.441429,313847.160000,11.471712,17.748147,35.274972,52.390881,0


In [0]:
# b. getting plant material combination which is between the range of 5% to 1% (less than 35 orders but greater than 10 orders) of order that has been placed.

ves_consump_v2=ves_consump[(ves_consump['order_counts']<35) & (ves_consump['order_counts']>=10)]
ves_consump_v2.drop(columns=['order_placed','order_counts'],inplace=True)
ves_consump_v2.drop_duplicates(inplace=True)
h_inv_ves_consump_fin1=pd.merge(h_inv_ves_consump,ves_consump_v2,on=['LocationID','MaterialID'],how='inner')
print(h_inv_ves_consump_fin1.shape)

#getting plant material combination for the second set
Rop02=h_inv_ves_consump_fin1[['LocationID','MaterialID']].copy()
Rop02.drop_duplicates(inplace=True)
print(Rop02.shape)

#second data set
h_inv_ves_consump_fin_v1=h_inv_ves_consump_fin1.drop(columns=['Plant','Material'])

print(h_inv_ves_consump_fin_v1.shape)
h_inv_ves_consump_fin1.head()

(84741, 22)
(128, 2)
(84741, 20)
Out[66]:

,SnapshotDate,MaterialID,LocationID,InventoryUnitOfMeasure,Week,InventoryStockUnRestricted_daily,Planned_avg_lead_time,Planned_GR_date,Extended_GR_date_2,Extended_GR_date_4,Extended_GR_date_6,Plant,Material,counter,Expected_consumption_btw_gr_dates_2,Expected_consumption_btw_gr_dates_4,Expected_consumption_btw_gr_dates_6,inventory_ratio_with_0,Expected_consumption_ratio_2_with_0,Expected_consumption_ratio_4_with_0,Expected_consumption_ratio_6_with_0,order_placed
0,2019-11-30,22000213,5446,LB,48,48350.00,28.0,2019-12-28,2020-01-11,2020-01-25,2020-02-08,5446,22000213,26,38089.565000,86154.534286,125188.781429,17.324857,13.648320,30.871043,44.857863,0
1,2019-12-01,22000213,5446,LB,48,48350.00,28.0,2019-12-29,2020-01-12,2020-01-26,2020-02-09,5446,22000213,27,39078.855000,87674.927143,125362.530714,17.324857,14.002804,31.415833,44.920121,0
2,2019-12-02,22000213,5446,LB,49,48350.00,28.0,2019-12-30,2020-01-13,2020-01-27,2020-02-10,5446,22000213,28,40068.145000,89195.320000,125536.280000,17.324857,14.357288,31.960623,44.982379,0
3,2019-12-03,22000213,5446,LB,49,48253.92,28.0,2019-12-31,2020-01-14,2020-01-28,2020-02-11,5446,22000213,29,40831.189286,89478.405714,125657.190000,17.290429,14.630703,32.062059,45.025704,0
4,2019-12-04,22000213,5446,LB,49,37960.54,28.0,2020-01-01,2020-01-15,2020-01-29,2020-02-12,5446,22000213,30,41594.233571,89761.491429,125778.100000,13.602087,14.904118,32.163495,45.069029,0


In [0]:
# overall plant material combination in h_inv_ves_consumption

Rop12=h_inv_ves_consump[['LocationID','MaterialID']].copy()
Rop12.drop_duplicates(inplace=True)
Rop12.shape

Out[67]: (400, 2)

In [0]:
#def function checkaccuracy is to find the best model suitable for a plant material combination where the number of orders placed is the closest to actual.
def checkaccuracy(dtrees):
  dtrees=dtrees.fit(X_train, Y_train)
  predictions=dtrees.predict(X_test)
  train=dtrees.score(X_train, Y_train)
  test=dtrees.score(X_test, Y_test)
  cm1 = confusion_matrix(Y_test,predictions)
  scores=abs(cm1[0,1]-cm1[1,0])
  return train,test,cm1,scores

In [0]:
# def model_creation is used to fit the best model for a plant material cobination
def model_creation(dtrees):
  dtrees=dtrees.fit(X_train, Y_train)
  predictions=dtrees.predict(X_test)
  cm1 = confusion_matrix(Y_test,predictions)
  return predictions,cm1

In [0]:
final_df=pd.DataFrame()
for plt_id,mt_id in zip(Rop12['LocationID'],Rop12['MaterialID']):
  
  #selecting first cut data for modelling
  if Rop01[(Rop01['LocationID']==plt_id) & (Rop01['MaterialID']==mt_id)].shape[0]>0:
    a=h_inv_ves_consump_fin_v2[(h_inv_ves_consump_fin_v2['LocationID']==plt_id) & (h_inv_ves_consump_fin_v2['MaterialID']==mt_id)]
    features=['counter','inventory_ratio_with_0','Expected_consumption_ratio_2_with_0', 'Expected_consumption_ratio_4_with_0', 'Expected_consumption_ratio_6_with_0']
    X=a[features]
    y=a['order_placed']
    
    # Train test split
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.3, random_state = 2)
    
    #1. Decision tree
    dtrees=DecisionTreeClassifier(criterion='entropy')
    train1,test1,cm1,scores1=checkaccuracy(dtrees)
    
    #2. Bagging classifier with base estimator as Decision tree
    dtrees=BaggingClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=18)
    train2,test2,cm2,scores2=checkaccuracy(dtrees)
    
    #3. BaggingClassifier
    dtrees=BaggingClassifier(n_estimators=2)
    train3,test3,cm3,scores3=checkaccuracy(dtrees)
    
    #4. Random forest classifier
    dtrees=RandomForestClassifier(n_estimators=30)
    train4,test4,cm4,scores4=checkaccuracy(dtrees)
    
    #5. Ada boost classifier with base estimator as Decision tree
    dtrees=AdaBoostClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=2)
    train5,test5,cm5,scores5=checkaccuracy(dtrees)
    
    #6. Ada boost classifier
    dtrees=AdaBoostClassifier(n_estimators=2)
    train6,test6,cm6,scores6=checkaccuracy(dtrees)
    
    #7. Gradient boosting classifier
    dtrees=GradientBoostingClassifier(n_estimators=2)
    train7,test7,cm7,scores7=checkaccuracy(dtrees)
    
    #8. KNeighbor classifier
    dtrees=KNeighborsClassifier(n_neighbors=20)
    train8,test8,cm8,scores8=checkaccuracy(dtrees)
    
    #creating tempdf to store all model values
    tempdf=pd.DataFrame(data=[[plt_id,mt_id,train1,test1,cm1[0,0],cm1[0,1],cm1[1,0],cm1[1,1],scores1],[plt_id,mt_id,train2,test2,cm2[0,0],cm2[0,1],cm2[1,0],cm2[1,1],scores2],[plt_id,mt_id,train3,test3,cm3[0,0],cm3[0,1],cm3[1,0],cm3[1,1],scores3],[plt_id,mt_id,train4,test4,cm4[0,0],cm4[0,1],cm4[1,0],cm4[1,1],scores4],[plt_id,mt_id,train5,test5,cm5[0,0],cm5[0,1],cm5[1,0],cm5[1,1],scores5],[plt_id,mt_id,train6,test6,cm6[0,0],cm6[0,1],cm6[1,0],cm6[1,1],scores6],[plt_id,mt_id,train7,test7,cm7[0,0],cm7[0,1],cm7[1,0],cm7[1,1],scores7],[plt_id,mt_id,train8,test8,cm8[0,0],cm8[0,1],cm8[1,0],cm8[1,1],scores8]],columns=['plant','material','train','test','cm[0,0]','cm[0,1]','cm[1,0]','cm[1,1]','score'],index=['DecisionTreeClassifier(criterion=entropy)','BaggingClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=18)','BaggingClassifier(n_estimators=2)','RandomForestClassifier(n_estimators=30)','AdaBoostClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=2)','AdaBoostClassifier(n_estimators=2)','GradientBoostingClassifier(n_estimators=2)','KNeighborsClassifier(n_neighbors=20)'])
    
    #selecting the best model out of score
    tempdf=tempdf.loc[tempdf['score'].idxmin()]
    final_df=final_df.append(tempdf)
    
  #selecting second cut data for modelling which needs fixing imbalanced classification before modelling
  elif Rop02[(Rop02['LocationID']==plt_id) & (Rop02['MaterialID']==mt_id)].shape[0]>0:
    a=h_inv_ves_consump_fin_v1[(h_inv_ves_consump_fin_v1['LocationID']==plt_id) & (h_inv_ves_consump_fin_v1['MaterialID']==mt_id)]
    features=['counter','inventory_ratio_with_0','Expected_consumption_ratio_2_with_0', 'Expected_consumption_ratio_4_with_0', 'Expected_consumption_ratio_6_with_0']
    X=a[features]
    y=a['order_placed']
    
    # SMOTEENN sampling (fixing imbalanced classification)
    os1 = SMOTEENN(enn=EditedNearestNeighbours(sampling_strategy='all'))
    X1=a[features]
    y1=a['order_placed']
    X1,y1=os1.fit_resample(X1,y1)
    
    # Train test split
    X_train, X_test, Y_train, Y_test = train_test_split(X1, y1, test_size = 0.3, random_state = 2)
    
    #1. Decision tree
    dtrees=DecisionTreeClassifier(criterion='entropy')
    train1,test1,cm1,scores1=checkaccuracy(dtrees)
    
    #2. Bagging classifier with base estimator as Decision tree
    dtrees=BaggingClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=18)
    train2,test2,cm2,scores2=checkaccuracy(dtrees)
    
    #3. BaggingClassifier
    dtrees=BaggingClassifier(n_estimators=2)
    train3,test3,cm3,scores3=checkaccuracy(dtrees)
    
    #4. Random forest classifier
    dtrees=RandomForestClassifier(n_estimators=30)
    train4,test4,cm4,scores4=checkaccuracy(dtrees)
    
    #5. Ada boost classifier with base estimator as Decision tree
    dtrees=AdaBoostClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=2)
    train5,test5,cm5,scores5=checkaccuracy(dtrees)
    
    #6. Ada boost classifier
    dtrees=AdaBoostClassifier(n_estimators=2)
    train6,test6,cm6,scores6=checkaccuracy(dtrees)
    
    #7. Gradient boosting classifier
    dtrees=GradientBoostingClassifier(n_estimators=2)
    train7,test7,cm7,scores7=checkaccuracy(dtrees)
    
    #8. KNeighbor classifier
    dtrees=KNeighborsClassifier(n_neighbors=20)
    train8,test8,cm8,scores8=checkaccuracy(dtrees)
    
    #creating tempdf to store all model values
    tempdf=pd.DataFrame(data=[[plt_id,mt_id,train1,test1,cm1[0,0],cm1[0,1],cm1[1,0],cm1[1,1],scores1],[plt_id,mt_id,train2,test2,cm2[0,0],cm2[0,1],cm2[1,0],cm2[1,1],scores2],[plt_id,mt_id,train3,test3,cm3[0,0],cm3[0,1],cm3[1,0],cm3[1,1],scores3],[plt_id,mt_id,train4,test4,cm4[0,0],cm4[0,1],cm4[1,0],cm4[1,1],scores4],[plt_id,mt_id,train5,test5,cm5[0,0],cm5[0,1],cm5[1,0],cm5[1,1],scores5],[plt_id,mt_id,train6,test6,cm6[0,0],cm6[0,1],cm6[1,0],cm6[1,1],scores6],[plt_id,mt_id,train7,test7,cm7[0,0],cm7[0,1],cm7[1,0],cm7[1,1],scores7],[plt_id,mt_id,train8,test8,cm8[0,0],cm8[0,1],cm8[1,0],cm8[1,1],scores8]],columns=['plant','material','train','test','cm[0,0]','cm[0,1]','cm[1,0]','cm[1,1]','score'],index=['DecisionTreeClassifier(criterion=entropy)','BaggingClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=18)','BaggingClassifier(n_estimators=2)','RandomForestClassifier(n_estimators=30)','AdaBoostClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=2)','AdaBoostClassifier(n_estimators=2)','GradientBoostingClassifier(n_estimators=2)','KNeighborsClassifier(n_neighbors=20)'])
    
    #selecting the best model out of score
    tempdf=tempdf.loc[tempdf['score'].idxmin()]
    final_df=final_df.append(tempdf)
    
  else:
    pass
  
final_df.reset_index(inplace=True)
final_df.display()

index,"cm[0,0]","cm[0,1]","cm[1,0]","cm[1,1]",material,plant,score,test,train
DecisionTreeClassifier(criterion=entropy),167.0,20.0,21.0,2.0,4.3647982E7,5259.0,1.0,0.8047619047619048,1.0
DecisionTreeClassifier(criterion=entropy),169.0,19.0,20.0,3.0,4.4002035E7,5259.0,1.0,0.8151658767772512,1.0
DecisionTreeClassifier(criterion=entropy),99.0,8.0,14.0,6.0,4.4060516E7,5259.0,6.0,0.8267716535433071,1.0
DecisionTreeClassifier(criterion=entropy),87.0,12.0,11.0,4.0,4.4082206E7,5259.0,1.0,0.7982456140350878,1.0
DecisionTreeClassifier(criterion=entropy),98.0,7.0,7.0,2.0,4.4082207E7,5259.0,0.0,0.8771929824561403,1.0
"AdaBoostClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=2)",78.0,20.0,17.0,4.0,4.4082212E7,5259.0,3.0,0.6890756302521008,1.0
DecisionTreeClassifier(criterion=entropy),146.0,14.0,11.0,171.0,2.2000213E7,5446.0,3.0,0.9269005847953217,1.0
BaggingClassifier(n_estimators=2),175.0,11.0,13.0,176.0,2.200117E7,5446.0,2.0,0.936,0.9794050343249427
DecisionTreeClassifier(criterion=entropy),188.0,12.0,13.0,1.0,2.2002227E7,5446.0,1.0,0.883177570093458,1.0
DecisionTreeClassifier(criterion=entropy),170.0,19.0,22.0,2.0,2.2002374E7,5446.0,3.0,0.8075117370892019,1.0


In [0]:
model_predicted=pd.DataFrame()
for plt_id,mt_id in zip(Rop12['LocationID'],Rop12['MaterialID']):
  
  #selecting first cut data for modelling
  if Rop01[(Rop01['LocationID']==plt_id) & (Rop01['MaterialID']==mt_id)].shape[0]>0:
    tempdf1=pd.DataFrame()
    a=h_inv_ves_consump_fin_v2[(h_inv_ves_consump_fin_v2['LocationID']==plt_id) & (h_inv_ves_consump_fin_v2['MaterialID']==mt_id)]
    features=['counter','inventory_ratio_with_0','Expected_consumption_ratio_2_with_0', 'Expected_consumption_ratio_4_with_0', 'Expected_consumption_ratio_6_with_0']
    X=a[features]
    y=a['order_placed']
    
    # Train test split
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.3, random_state = 2)
    
    # building the model based on the best chosen model for a plant material combination  
    model=final_df[(final_df['plant']==plt_id) & (final_df['material']==mt_id)]['index'].values[0]  
    if model =='DecisionTreeClassifier(criterion=entropy)':
      dtrees=DecisionTreeClassifier()
      predictions,cm1=model_creation(dtrees)
    elif model =='BaggingClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=18)':
      param_grid={'base_estimator':['DecisionTreeClassifier()'],'n_estimators':[18]}
      dtrees=BaggingClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=18)
      predictions,cm1=model_creation(dtrees)
    elif model =='BaggingClassifier(n_estimators=2)':
      dtrees=BaggingClassifier(n_estimators=2)
      predictions,cm1=model_creation(dtrees)
    elif model =='RandomForestClassifier(n_estimators=30)':
      dtrees=RandomForestClassifier(n_estimators=30)
      predictions,cm1=model_creation(dtrees)
    elif model =='AdaBoostClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=2)':
      dtrees=AdaBoostClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=2)
      predictions,cm1=model_creation(dtrees)
    elif model =='AdaBoostClassifier(n_estimators=2)':
      dtrees=AdaBoostClassifier(n_estimators=2)
      predictions,cm1=model_creation(dtrees)
    elif model =='GradientBoostingClassifier(n_estimators=2)':
      dtrees=GradientBoostingClassifier(n_estimators=2)
      predictions,cm1=model_creation(dtrees)
    elif model =='KNeighborsClassifier(n_neighbors=20)':
      dtrees=KNeighborsClassifier(n_neighbors=20)
      predictions,cm1=model_creation(dtrees)
    else:
      print('model_not_built')
    tempdf=X_test.copy()
    tempdf['LocationID']=plt_id
    tempdf['MaterialID']=mt_id
    tempdf['order_placed']=Y_test
    tempdf['predicted_order_placed']=predictions
    tempdf1=a[['LocationID','MaterialID']].copy()
    tempdf1.drop_duplicates(inplace=True)
    tempdf1['cm[0,0]']=cm1[0,0]
    tempdf1['cm[0,1]']=cm1[0,1]
    tempdf1['cm[1,0]']=cm1[1,0]
    tempdf1['cm[1,1]']=cm1[1,1]
    tempdf2=pd.merge(tempdf,tempdf1,on=['LocationID','MaterialID'],how='left')
    model_predicted=model_predicted.append(tempdf2)

    # save the model to disk
    print(plt_id,mt_id,dtrees)
    root_directory = '/dbfs/FileStore/models/classifiers/'
    directory = f"{model_run_date}"+'/'+str(plt_id)+'/'+str(mt_id)+'/'
    path = os.path.join(root_directory, directory)
    os.makedirs(path, exist_ok=True)
    file_name = path + '_class_if.pkl'
    pickle.dump(dtrees, open(file_name, "wb"))
   
  #selecting second cut data for modelling
  elif Rop02[(Rop02['LocationID']==plt_id) & (Rop02['MaterialID']==mt_id)].shape[0]>0:
    tempdf1=pd.DataFrame()
    a=h_inv_ves_consump_fin_v1[(h_inv_ves_consump_fin_v1['LocationID']==plt_id) & (h_inv_ves_consump_fin_v1['MaterialID']==mt_id)]
    features=['counter','inventory_ratio_with_0','Expected_consumption_ratio_2_with_0', 'Expected_consumption_ratio_4_with_0', 'Expected_consumption_ratio_6_with_0']
    X=a[features]
    y=a['order_placed']
    
    # SMOTEENN sampling (fixing imbalanced classification)
    os1 = SMOTEENN(enn=EditedNearestNeighbours(sampling_strategy='all'))
    X1=a[features]
    y1=a['order_placed']
    X1,y1=os1.fit_resample(X1,y1)
    
    # Train test split
    X_train, X_test, Y_train, Y_test = train_test_split(X1, y1, test_size = 0.3, random_state = 2)
    
    # building the model based on the best chosen model for a plant material combination  
    model=final_df[(final_df['plant']==plt_id) & (final_df['material']==mt_id)]['index'].values[0]  
    if model =='DecisionTreeClassifier(criterion=entropy)':
      dtrees=DecisionTreeClassifier()
      predictions,cm1=model_creation(dtrees)
    elif model =='BaggingClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=18)':
      param_grid={'base_estimator':['DecisionTreeClassifier()'],'n_estimators':[18]}
      dtrees=BaggingClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=18)
      predictions,cm1=model_creation(dtrees)
    elif model =='BaggingClassifier(n_estimators=2)':
      dtrees=BaggingClassifier(n_estimators=2)
      predictions,cm1=model_creation(dtrees)
    elif model =='RandomForestClassifier(n_estimators=30)':
      dtrees=RandomForestClassifier(n_estimators=30)
      predictions,cm1=model_creation(dtrees)
    elif model =='AdaBoostClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=2)':
      dtrees=AdaBoostClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=2)
      predictions,cm1=model_creation(dtrees)
    elif model =='AdaBoostClassifier(n_estimators=2)':
      dtrees=AdaBoostClassifier(n_estimators=2)
      predictions,cm1=model_creation(dtrees)
    elif model =='GradientBoostingClassifier(n_estimators=2)':
      dtrees=GradientBoostingClassifier(n_estimators=2)
      predictions,cm1=model_creation(dtrees)
    elif model =='KNeighborsClassifier(n_neighbors=20)':
      dtrees=KNeighborsClassifier(n_neighbors=20)
      predictions,cm1=model_creation(dtrees)
    else:
      print('model_not_built')
    tempdf=X_test.copy()
    tempdf['LocationID']=plt_id
    tempdf['MaterialID']=mt_id
    tempdf['order_placed']=Y_test
    tempdf['predicted_order_placed']=predictions
    tempdf1=a[['LocationID','MaterialID']].copy()
    tempdf1.drop_duplicates(inplace=True)
    tempdf1['cm[0,0]']=cm1[0,0]
    tempdf1['cm[0,1]']=cm1[0,1]
    tempdf1['cm[1,0]']=cm1[1,0]
    tempdf1['cm[1,1]']=cm1[1,1]
    tempdf2=pd.merge(tempdf,tempdf1,on=['LocationID','MaterialID'],how='left')
    model_predicted=model_predicted.append(tempdf2)

    # save the model to disk
    print(plt_id,mt_id,dtrees)
    root_directory = '/dbfs/FileStore/models/classifiers/'
    directory = f"{model_run_date}"+'/'+str(plt_id)+'/'+str(mt_id)+'/'
    path = os.path.join(root_directory, directory)
    os.makedirs(path, exist_ok=True)
    file_name = path + '_class_if.pkl'
    pickle.dump(dtrees, open(file_name, "wb"))
    
  else:
    print('unable to build model')
  
model_predicted.display()

unable to build model
5259 43647982 DecisionTreeClassifier()
unable to build model
unable to build model
unable to build model
5259 44002035 DecisionTreeClassifier()
5259 44060516 DecisionTreeClassifier()
5259 44082206 DecisionTreeClassifier()
5259 44082207 DecisionTreeClassifier()
5259 44082212 AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=2)
5446 22000213 DecisionTreeClassifier()
5446 22001170 BaggingClassifier(n_estimators=2)
unable to build model
5446 22002227 DecisionTreeClassifier()
unable to build model
5446 22002374 DecisionTreeClassifier()
5446 22002618 RandomForestClassifier(n_estimators=30)
5446 22002629 DecisionTreeClassifier()
5446 22002632 AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=2)
5446 22006408 BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=18)
5959 22002233 DecisionTreeClassifier()
5959 22002248 DecisionTreeClassifier()
5959 22002374 BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=18)
5959 22002379 AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=2)
5959 22002410 DecisionTreeClassifier()
5959 22002411 AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=2)
5959 22002475 AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=2)
5959 22002512 DecisionTreeClassifier()
5959 22002514 DecisionTreeClassifier()
5959 22002525 DecisionTreeClassifier()
5959 22002545 AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=2)
5959 22002560 DecisionTreeClassifier()
5959 22002573 BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=18)
5959 22002618 AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=2)
5959 22002629 DecisionTreeClassifier()
5959 22002679 DecisionTreeClassifier()
5258 43058765 AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=2)
5258 43534638 DecisionTreeClassifier()
5258 43559239 DecisionTreeClassifier()
5258 43572559 DecisionTreeClassifier()
5258 43642252 AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=2)
unable to build model
5258 43646018 AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=2)
5258 43646139 AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=2)
5258 43646226 AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=2)
unable to build model
5258 43941480 AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=2)
unable to build model
unable to build model
unable to build model
unable to build model
unable to build model
unable to build model
5258 44082209 AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=2)
5258 44082213 DecisionTreeClassifier()
5446 43512661 DecisionTreeClassifier()
5446 43533466 AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=2)
5446 43548623 DecisionTreeClassifier()
5446 43549260 DecisionTreeClassifier()
5446 43641218 DecisionTreeClassifier()
unable to build model
5446 43748707 AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=2)
5446 43851043 DecisionTreeClassifier()
5446 44034197 AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=2)
unable to build model
unable to build model
unable to build model
5878 22000873 BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=18)
5878 22001236 AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=2)
unable to build model
5878 22001851 BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=18)
5878 22002194 RandomForestClassifier(n_estimators=30)
5878 22002216 DecisionTreeClassifier()
5878 22002218 RandomForestClassifier(n_estimators=30)
5878 22002222 BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=18)
5878 22002227 DecisionTreeClassifier()
5959 22001170 AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estim

counter,inventory_ratio_with_0,Expected_consumption_ratio_2_with_0,Expected_consumption_ratio_4_with_0,Expected_consumption_ratio_6_with_0,LocationID,MaterialID,order_placed,predicted_order_placed,"cm[0,0]","cm[0,1]","cm[1,0]","cm[1,1]"
1,4.830704913479127,17.63600559460922,34.44807590807403,48.53732677934981,5259,43647982,0,1,161,26,21,2
3,7.114340313170395,15.835155740682955,28.118648593468798,46.31923563040462,5259,43647982,0,0,161,26,21,2
33,11.695816956992203,12.885042519183262,25.65869149623463,39.25289706710433,5259,43647982,0,0,161,26,21,2
26,3.1146522597100588,14.492852337832899,26.4647817137176,41.08747301345221,5259,43647982,0,0,161,26,21,2
9,7.214098393510121,12.474243933519604,27.415103223530316,38.384621045794376,5259,43647982,0,0,161,26,21,2
6,12.012452041979477,15.88884602863018,30.091333123924418,41.26072338572627,5259,43647982,1,0,161,26,21,2
3,12.032800954284383,16.752537642461537,34.31419578048709,51.24942900053924,5259,43647982,0,0,161,26,21,2
2,8.786256359774429,16.424715931924595,28.39766680746777,46.14581331900279,5259,43647982,0,0,161,26,21,2
2,10.182084907928205,17.509745160517735,34.36893740704338,48.352417455680126,5259,43647982,0,0,161,26,21,2
4,8.171882800734382,20.010349138539016,36.56931492083169,54.24241036080876,5259,43647982,0,0,161,26,21,2


In [0]:
model_predicted.display()

counter,inventory_ratio_with_0,Expected_consumption_ratio_2_with_0,Expected_consumption_ratio_4_with_0,Expected_consumption_ratio_6_with_0,LocationID,MaterialID,order_placed,predicted_order_placed,"cm[0,0]","cm[0,1]","cm[1,0]","cm[1,1]"
1,4.830704913479127,17.63600559460922,34.44807590807403,48.53732677934981,5259,43647982,0,1,161,26,21,2
3,7.114340313170395,15.835155740682955,28.118648593468798,46.31923563040462,5259,43647982,0,0,161,26,21,2
33,11.695816956992203,12.885042519183262,25.65869149623463,39.25289706710433,5259,43647982,0,0,161,26,21,2
26,3.1146522597100588,14.492852337832899,26.4647817137176,41.08747301345221,5259,43647982,0,0,161,26,21,2
9,7.214098393510121,12.474243933519604,27.415103223530316,38.384621045794376,5259,43647982,0,0,161,26,21,2
6,12.012452041979477,15.88884602863018,30.091333123924418,41.26072338572627,5259,43647982,1,0,161,26,21,2
3,12.032800954284383,16.752537642461537,34.31419578048709,51.24942900053924,5259,43647982,0,0,161,26,21,2
2,8.786256359774429,16.424715931924595,28.39766680746777,46.14581331900279,5259,43647982,0,0,161,26,21,2
2,10.182084907928205,17.509745160517735,34.36893740704338,48.352417455680126,5259,43647982,0,0,161,26,21,2
4,8.171882800734382,20.010349138539016,36.56931492083169,54.24241036080876,5259,43647982,0,0,161,26,21,2


In [0]:
plt_id=5004
mt_id=22000848
df_1=pd.DataFrame(data=[[45,
10.725555,
16.353074,
32.952447,
49.889764]],columns=['counter', 'inventory_ratio_with_0', 'Expected_consumption_ratio_2_with_0', 'Expected_consumption_ratio_4_with_0', 'Expected_consumption_ratio_6_with_0'])

In [0]:
model_run_date='2022-03-25'
file_name = '/dbfs/FileStore/models/classifiers/'+f"{model_run_date}"+'/'+str(plt_id)+'/'+str(mt_id)+'/'+ '_class_if.pkl'
loaded_model = pickle.load(open(file_name, "rb"))
reorder_pred = loaded_model.predict(df_1)[0]

In [0]:
reorder_pred

Out[75]: 1

**END